In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import time
import plotly.graph_objs as go

## Import Tide Height Measurement Data

In [ ]:
feed_a_pubkey = "95qr2ub6pbyx" # modify this to match a public key on belfast.pvos.org

base_url = "http://belfast.pvos.org/data/"

start = time.time()
# dfm = pd.read_csv(base_url+feed_a_pubkey+"/csv/", index_col='timestamp', parse_dates=True)
dfm = pd.read_csv("tide_data_95qr2ub6pbyx.csv", index_col='datetime', parse_dates=True)

end = time.time()
#dfm.to_csv("tide_data_" + feed_a_pubkey + ".csv", index=False)
# dfm.to_csv("tide_data_" + feed_a_pubkey + ".csv")

print("download took: ", end-start, " seconds")

In [ ]:
# dfm.to_csv("tide_data_" + feed_a_pubkey + ".csv")

In [ ]:
dfm.columns

Import Divirod Measurement Data

In [ ]:
dfm.index[0]

In [ ]:
dfm.index = pd.to_datetime(dfm.index,unit='s')
dfm.index = dfm.index.tz_localize('UTC')


In [ ]:
dfm.index[-1]

In [ ]:
dfm = dfm.sort_index()
dfm.index[-1]

In [ ]:
dfm.index[0]

## Import tide predictions from NOAA

In [ ]:
# START_TIMESTAMP = '2021-10-01 6:00:00'
START_TIMESTAMP = dfm.index[0]
START_DATE = dfm.index[0].date().strftime('%Y%m%d')
END_TIMESTAMP = dfm.index[-1]
END_DATE = dfm.index[-1].date().strftime('%Y%m%d')
END_DATE

## Use an estimate of the sensor height above MLLW to convert Distance to Tide Height

In [ ]:
dfm["UltrasonicRadarDifference"] = dfm['distance_mm'] -  dfm['distance_meters_1']
est_us_sensor_height_abv_MLLW_ft = 17.84
est_radar_sensor_height_abv_MLLW_ft = 17.84 - (82.96/304.8)  # 17.56782152230971

dfm["us_height_ft"] = est_us_sensor_height_abv_MLLW_ft - dfm['distance_mm'] / 304.8
dfm["radar_height_ft"] = est_radar_sensor_height_abv_MLLW_ft - dfm['distance_meters_1'] / 304.8




In [ ]:
dfm["radar_mm"] = dfm['distance_meters_1']
dfm["ultrasonic_mm"] = dfm['distance_mm']
dfur_mm = dfm.filter(["radar_mm","ultrasonic_mm"], axis=1)

In [ ]:
dfur_mm.to_csv("Belfast_Ultrasonic_and_Radar_mm_2021_12_14" + ".csv")

In [ ]:
# dfur_mm = pd.read_csv("Belfast_Ultrasonic_and_Radar_mm_2021_12_14" + ".csv")

In [ ]:
dfur_mm.info()

In [ ]:
dfur_mm.head()

## Create a subset containing just the estimated Tide Height with The Start-End time Range

In [ ]:
c0 = dfm.index.to_series().between(START_TIMESTAMP,END_TIMESTAMP)


#create a new dataframe using this index
height_us=dfm[c0]["us_height_ft"]
height_radar=dfm[c0]["radar_height_ft"]
height_delta_radar_us = dfm[c0]["UltrasonicRadarDifference"]


In [ ]:
station = "8415191" # Belfast, Maine
start_date = "20211001"
end_date = END_DATE
timezone = "gmt"
units = "english"
noaa_url = "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?" +\
            f"begin_date={start_date}" +\
            f"&end_date={end_date}" +\
            f"&station={station}" +\
            "&product=predictions" +\
            "&datum=MLLW" +\
            f"&time_zone={timezone}" +\
            "&interval=hilo" +\
            f"&units={units}" +\
            "&format=csv"
start = time.time()
dfp = pd.read_csv(noaa_url, index_col='Date Time', parse_dates=True)
end = time.time()

print("download took: ", end-start, " seconds")
dfp.index = dfp.index.tz_localize('UTC')
dfp.index[-1]


In [ ]:
dfp.index[0]
dfp.head()

## Resample height measurements to 6 minute interval

In [ ]:

height_us_6m = height_us.resample('6min').mean()
height_radar_6m = height_radar.resample('6min').mean()
height_delta_1m = height_delta_radar_us.resample('1min').mean()

height_us_6m.to_csv("us_tide_height_6min_02Oct2021_to_09Oct2021.csv")
height_radar_6m.to_csv("radar_tide_height_6min_02Oct2021_to_09Oct2021.csv")

height_us_6m.head()
height_radar_6m.head()
height_delta_1m.head()

In [ ]:
height_delta_1m

In [ ]:
height_ur_6m = height_us.resample('6min').mean()
# height_radar_6m = height_radar.resample('6min').mean()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
  x=height_ur_6m.index.tz_convert('America/New_York'), 
  y=height_ur_6m,
  mode='lines', 
  connectgaps= False,
  name='Ultrasonic',
  ))

fig.add_trace(go.Scatter(
  x=height_radar_6m.index.tz_convert('America/New_York'), 
  y=height_radar_6m,
  mode='lines', 
  connectgaps= False,
  name='radar',
  ))

fig.update_layout(
    title="Ultrasonic and Radar Tide Height, ft above MLLW",
)

fig.show()

## Plot predicted and measured tides

In [ ]:
# fig = go.Figure()

# fig.add_trace(go.Scatter(
#   x=height_delta_1m.index.tz_convert('America/New_York'), 
#   y=height_delta_1m,
#   mode='lines', 
#   connectgaps= False,
#   name='delta',
#   ))

# fig.update_layout(
#     title="Ultrasonic Measurement minus Radar Measurement, mm",
# )

# fig.show()

In [ ]:
# plotly setup and traces

colorsIdx = {'H': 'black', 'L': 'black'}
cols      = dfp[' Type'].map(colorsIdx)

fig = go.Figure()

fig.add_trace(go.Scatter(
  x=dfp.index.tz_convert('America/New_York'), 
  y=dfp[' Prediction'],
  mode='markers', 
  connectgaps= True,
  name='predictions',
  marker=dict(size=6, opacity=0.5,color=cols),
  ))

fig.add_trace(go.Scatter(
  x=height_us_6m.index.tz_convert('America/New_York'), 
  y=height_us_6m.values,
  mode='lines', 
  connectgaps= True,
  name='Ultrasonic',
  line = dict(color='green', width=1),
  ))

fig.add_trace(go.Scatter(
  x=height_radar_6m.index.tz_convert('America/New_York'), 
  y=height_radar_6m.values,
  mode='lines', 
  connectgaps= True,
  name='Radar',
  line = dict(color='red', width=1),
  ))


fig.update_layout(
    title="Predicted vs US and Radar 5 Minute Mean Resample",
    height=600,
)

  
fig.show()



Resample Height Measurements to 6 Second interval and Plot

In [ ]:
## Plot all samples of Height Measurements (6 per second)

In [ ]:
# colorsIdx = {'H': 'blue', 'L': 'orange'}
# cols      = dfp[' Type'].map(colorsIdx)

# fig = go.Figure()

# fig.add_trace(go.Scatter(
#   x=dfp.index.tz_convert('America/New_York'), 
#   y=dfp[' Prediction'],
#   mode='markers', 
#   connectgaps= True,
#   name='predictions',
#   marker=dict(size=6, opacity=0.5,color=cols),
#   ))

# fig.add_trace(go.Scatter(
#   x=height_us.index.tz_convert('America/New_York'), 
#   y=height_us.values,
#   mode='lines', 
#   connectgaps= False,
#   name='Ultrasonic',
#   line = dict(color='green', width=1),
#   ))

# fig.add_trace(go.Scatter(
#   x=height_radar.index.tz_convert('America/New_York'), 
#   y=height_radar.values,
#   mode='lines', 
#   connectgaps= False,
#   name='Radar',
#   line = dict(color='red', width=1),
#   ))

# fig.update_layout(
#     title="Predicted vs Measurements 6 Per Second Samples",
# )
  
# fig.show()

## Use "describe" aggregration to compute statistics over a time interval